In [3]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 15.5 MB/s eta 0:00:00


In [1]:
import numpy as np
np.set_printoptions(threshold=np.inf)

In [61]:
#Code du prof
import random
import numpy as np
import pulp
from sympy import *

def get_nash(J):
        lines=range(len(J))
        columns=range(len(J[0]))

        lp = pulp.LpProblem('Nash', pulp.LpMaximize)
        V = pulp.LpVariable('V', cat='Continuous')

        a = pulp.LpVariable.dicts("a", (lines),  cat="Continous")
        lp.setObjective(V)

        lp += pulp.lpSum([a[i] for i in lines]) == 1

        for i in lines :
            lp +=a[i]>=0

        for j in columns :
            lp += pulp.lpSum([a[i]*J[i][j] for i in lines])-V>=0


        lp.solve()

        return [pulp.value(lp.objective), [pulp.value(a[i]) for i in lines]]

m=4
J=np.zeros((m)*(m))
J.resize((m, m))
for i in range(m):
        for j in range(m):
            if i==j :
                J[i][j]=5
            else:
                J[i][j]=-abs((i-j))

qq=get_nash(J)
print(qq)

[-0.041666667, [0.29166667, 0.20833333, 0.20833333, 0.29166667]]


In [5]:
qq[1].index(max(qq[1]))

0



---

GAME

In [73]:
NB_CASES = 7
NB_PIERRES = 15

In [74]:
class Joueur:

  def __init__(self, nom, nb_pierres, position):
        self.name = nom
        self.nb_pierres = nb_pierres
        self.position = position

  def getName(self):
        return self.name
  
  def getNbPierres(self):
        return self.nb_pierres
  
  def jettePierres(self, n):
        if n > self.nb_pierres:
            self.nb_pierres = 0
        else:
            self.nb_pierres -= n
  
  def aZeroPierres(self):
        return self.nb_pierres <= 0
  
  def infos(self):
        return [self.name, self.nb_pierres]
  
  def getPosition(self):
        return self.position


In [75]:
class Troll:
  def __init__(self, position_troll):
        self.position = position_troll
  
  def getPosition(self):
        return self.position

  def moveTroll(self, n1, n2):
        if n1 > n2:
            self.position += 1
        elif n1 < n2:
            self.position -= 1

  def moveTrollEnd(self, j1, j2, n):
        if j1 > j2:
          self.position += n
        elif j1 < j2:
          self.position -= n
  
  def toString(self):
        return ["Le troll", self.position]



In [92]:
nbpj2 = 1
class Game:
  def __init__(self, player1, troll, player2):
        self.player1 = player1
        self.troll = troll
        self.player2 = player2
  
  def isOver(self):
        # le troll a atteint le chateau du joueur 1
        if self.troll.getPosition() == self.player1.getPosition():
            print("Le Troll à atteint le chateau de " + self.player1.getName())
            return True

        # le troll a atteint le chateau du joueur 2
        if self.troll.getPosition() == self.player2.getPosition():
            print("Le Troll à atteint le chateau de " + self.player2.getName())
            return True

        # les 2 joueurs n'ont plus aucune pierre à jeter
        if self.player1.aZeroPierres() and self.player2.aZeroPierres():
            
            # on regarde ou est la coordonnée du troll
            if self.troll.getPosition() == ((NB_CASES//2)+1):
                print("Les 2 joueurs n'ont plus aucune pierre à jeter: le Troll est au milieu. Egalité parfaite")
            elif self.troll.getPosition() < ((NB_CASES//2)+1):
                print("Les 2 joueurs n'ont plus aucune pierre à jeter. Le Troll est proche du chateau de " + self.player1.getName())
            else:
                print("Les 2 joueurs n'ont plus aucune pierre à jeter. Le Troll est proche du chateau de " + self.player2.getName())

            return True

        return False

  
  def startRound(self):
        # on vérifie si l'un des 2 joueurs n'a plus de pierre: si oui, le jeu se termine automatiquement
        if self.player1.aZeroPierres():
            # le joueur 1 n'a plus de pierres
            stone1 = 0
            stone2 = self.player2.getNbPierres()
            self.player2.jettePierres(stone2)
            self.troll.moveTrollEnd(stone1, stone2, stone2)
            print(self.player1.getName() + " n'a plus de pierres: " + self.player2.getName() + " va donc jeter ses pierres")
            stone1 = 0
            stone2 = 0
        elif self.player2.aZeroPierres():
            # le joueur 2 n'a plus de pierres
            stone1 = self.player1.getNbPierres()
            stone2 = 0
            self.player1.jettePierres(stone1)
            self.troll.moveTrollEnd(stone1, stone2, stone1)
            print(self.player2.getName() + " n'a plus de pierres: " + self.player1.getName() + " va donc jeter ses pierres")
            stone1 = 0
            stone2 = 0
        else:
            # chacun des joueurs choisit sa stratégie

            ''' cette partie de choisir le nombre de pierres est a modifier par des stratégies '''
            #matriceJ1 = np.zeros((self.player1.getNbPierres())*(self.player2.getNbPierres())*(self.troll.getPosition()))
            #matriceJ1.resize((self.player1.getNbPierres(), self.player2.getNbPierres(), self.troll.getPosition()))
            qq_J1 = get_nash(J[self.player1.getNbPierres()][self.player2.getNbPierres()])
            print(qq_J1)
            stone1 = qq_J1[1].index(max(qq_J1[1])) + 1

            #matriceJ2 = np.zeros((self.player2.getNbPierres())*(self.player1.getNbPierres())*(NB_CASES))
            #matriceJ2.resize((self.player2.getNbPierres(), self.player1.getNbPierres(), NB_CASES))
            #qq_J2 = get_nash([self.player2.getNbPierres()][self.player1.getNbPierres()])
            #stone2 = qq_J2[1].index(max(qq_J2[1])) + 1
            stone2 = 


            #qqJ1 = getVal_nash(self.player1.getNbPierres(), self.player2.getNbPierres(), self.troll.getPosition())
            #print(qqJ1)
            #if isinstance(qqJ1, int):
              #stone1 = 1
            #stone1 = qqJ1[1].index(max(qqJ1[1])) + 1
            #qqJ2 = getVal_nash(self.player2.getNbPierres(), self.player1.getNbPierres(), self.troll.getPosition())
            #stone2 = qqJ2[1].index(max(qqJ2[1])) + 1

            print(self.player1.getName() + " jette " + str(stone1) + " pierres contre " + self.player2.getName() + " qui jette " + str(stone2) + " pierres")
             
        self.player1.jettePierres(stone1)
        self.player2.jettePierres(stone2)
        self.troll.moveTroll(stone1, stone2)

  
  def toString(self):
        return [self.player1.toString(), self.troll.toString(), self.player2.toString()]
  
  def stats(self):
        return [self.player1.getNbPierres(), self.player2.getNbPierres(), self.troll.getPosition()]






In [86]:
#nb de pierres
NB_PIERRES_J1 = 15
NB_PIERRES_J2 = 15

#nb de cases
NB_CASES = 7 

POSITION_TROLL = 0

def main(): 
  # création des joueurs
  p1 = Joueur("Joueur 1", NB_PIERRES_J1, -(NB_CASES // 2))
  p2 = Joueur("Joueur 2", NB_PIERRES_J2, (NB_CASES // 2))

  # création du troll
  troll = Troll(POSITION_TROLL)

  # création de la partie
  game = Game(p1, troll, p2)

  #Affichage du statut du jeu
  print(game.stats())

  #Boucle d'exécution du jeu
  while not game.isOver():
        game.startRound()
        print(game.stats())



In [93]:
main()

[15, 15, 0]


IndexError: ignored



---

STRATEGY

In [107]:
#Dictionnaire pour stocker les valeurs de gains déjà connues
dictionnaireGame = {}

#Fonction pour ajouter un élément à un dictionnaire
#exemple : ajouter_element(dictionnairevaleur, (15,15,0), 0)
#ajouter dans le dictionnairevaleur un élément avec la clé (15,15,0) et pour valeur 0
def add_element(dictionnaire, cle, valeur):
    dictionnaire[cle] = valeur

#Fonction pour vérifier si la valeur en paramètre est présente dans le dictionnaire
#retourne vrai si elle est présente
#retourne faux sinon
def verify_element(dictionnaire, valeur):
    return valeur in dictionnaire

In [101]:
#Fonction pour obtenir le gain d'un coup possible en cours
def getVal(x,y,t):
    
    #Test pour vérifier si le troll est sur l'un des deux chateaux
    if t == (NB_CASES // 2):
      return 1
    elif t == -(NB_CASES // 2):
      return -1
    elif t == 0:
      if x==y:
        return 0
    
    # Matrice V(0,y,t)
    if x==0:
        if t >= 0:
            if y > t:
                 add_element(dictionnaireGame, (x,y,t), -1)
            elif y == t:
                 add_element(dictionnaireGame, (x,y,t), 0)
            elif y<t:
                add_element(dictionnaireGame, (x,y,t), 1)
        else:
         add_element(dictionnaireGame, (x,y,t), -1)
    
    # Matrice V(x,0,t)
    if y == 0:
         if t<=0:
             if x > abs(t):
                add_element(dictionnaireGame, (x,y,t), 1)
             elif x == abs(t):
                add_element(dictionnaireGame, (x,y,t), 0)
             elif x < abs(t):
              add_element(dictionnaireGame, (x,y,t), -1)
         else:
              add_element(dictionnaireGame, (x,y,t), 1)
    
    #On recherche si la valeur est deja dans le dictionnaire
    v = (x,y,t)
    if verify_element(dictionnaireGame,v):
        return dictionnaireGame[v]

    #Création d'une sous matrice
    SJ=np.zeros((x)*(y))
    SJ.resize((x, y))

    for i in range(x):
        for j in range(y):
          newX = x-1-i
          newY = y-1-j
          t2 = getT(x, y, newX, newY, t)

          #Si valeur deja dans le dictionnaire
          if verify_element(dictionnaireGame,(newX,newY,t2)):
              SJ[i][j] = dictionnaireGame[(newX,newY,t2)]
          #Sinon on calcule la valeur du sous jeu
          else: 
              SJ[i][j] = getVal(newX,newY,t2)
              add_element(dictionnaireGame, (newX,newY,t2), SJ[i][j])

    qq=get_nash(SJ)
    print("Matrice [",x,",",y,",",t,"]")
    print(qq)
    return qq[0]


#Fonction pour avoir la nouvelle position du troll 
def getT(nb_pierres_initial_j1, nb_pierres_initial_j2, nb_pierres_j1, nb_pierres_j2, oldT):
    differencepierreJ1 = nb_pierres_initial_j1 - nb_pierres_j1 
    differencepierreJ2 = nb_pierres_initial_j2 - nb_pierres_j2

    if(differencepierreJ1 == differencepierreJ2):
        newT = oldT
    elif(differencepierreJ1 < differencepierreJ2):
        newT = oldT - 1 
    else:
        newT = oldT + 1 
    return newT


In [81]:
def getVal_nash(x,y,t):
    
    #Test pour vérifier si le troll est sur l'un des deux chateaux
    if t == (NB_CASES // 2):
      return 1
    elif t == -(NB_CASES // 2):
      return -1
    elif t == 0:
      if x==y:
        return 0
    
    # Matrice V(0,y,t)
    if x==0:
        if t >= 0:
            if y > t:
                 add_element(dictionnaireGame, (x,y,t), -1)
            elif y == t:
                 add_element(dictionnaireGame, (x,y,t), 0)
            elif y<t:
                add_element(dictionnaireGame, (x,y,t), 1)
        else:
         add_element(dictionnaireGame, (x,y,t), -1)
    
    # Matrice V(x,0,t)
    if y == 0:
         if t<=0:
             if x > abs(t):
                add_element(dictionnaireGame, (x,y,t), 1)
             elif x == abs(t):
                add_element(dictionnaireGame, (x,y,t), 0)
             elif x < abs(t):
              add_element(dictionnaireGame, (x,y,t), -1)
         else:
              add_element(dictionnaireGame, (x,y,t), 1)
    
    #On recherche si la valeur est deja dans le dictionnaire
    v = (x,y,t)
    if verify_element(dictionnaireGame,v):
        return dictionnaireGame[v]

    #Création d'une sous matrice
    SJ=np.zeros((x)*(y))
    SJ.resize((x, y))

    for i in range(x):
        for j in range(y):
          newX = x-1-i
          newY = y-1-j
          t2 = getT(x, y, newX, newY, t)

          #Si valeur deja dans le dictionnaire
          if verify_element(dictionnaireGame,(newX,newY,t2)):
              SJ[i][j] = dictionnaireGame[(newX,newY,t2)]
          #Sinon on calcule la valeur du sous jeu
          else: 
              SJ[i][j] = getVal(newX,newY,t2)
              add_element(dictionnaireGame, (newX,newY,t2), SJ[i][j])

    qq=get_nash(SJ)
    #print(qq)
    return qq

In [89]:
get_nash(J[15][15])

IndexError: ignored

In [83]:
# EXEMPLE MAIN 

NB_PIERRES_J1 = 15
NB_PIERRES_J2 = 15
POSITION_TROLL = 0
print(getVal(NB_PIERRES_J1,NB_PIERRES_J2,POSITION_TROLL))

0


In [14]:
# EXEMPLE MAIN 

NB_PIERRES_J1 = 23
NB_PIERRES_J2 = 17
POSITION_TROLL = -1
print(getVal(NB_PIERRES_J1,NB_PIERRES_J2,POSITION_TROLL))

0.49907472


In [ ]:
# EXEMPLE MAIN 

NB_PIERRES_J1 = 50
NB_PIERRES_J2 = 30
POSITION_TROLL = -2
print(getVal(NB_PIERRES_J1,NB_PIERRES_J2,POSITION_TROLL))

In [108]:
# EXEMPLE MAIN 

NB_PIERRES_J1 = 23
NB_PIERRES_J2 = 21
POSITION_TROLL = -1
print(getVal(NB_PIERRES_J1,NB_PIERRES_J2,POSITION_TROLL))

Matrice [ 3 , 1 , -1 ]
[1.0, [0.0, 1.0, 0.0]]
Matrice [ 2 , 1 , 0 ]
[1.0, [1.0, 0.0]]
Matrice [ 4 , 2 , -1 ]
[1.0, [1.0, 0.0, 0.0, 0.0]]
Matrice [ 4 , 1 , -2 ]
[1.0, [0.0, 1.0, 0.0, 0.0]]
Matrice [ 1 , 1 , 1 ]
[1.0, [1.0]]
Matrice [ 3 , 2 , 0 ]
[1.0, [1.0, 0.0, 0.0]]
Matrice [ 1 , 2 , 0 ]
[-1.0, [1.0]]
Matrice [ 5 , 3 , -1 ]
[1.0, [1.0, 0.0, 0.0, 0.0, 0.0]]
Matrice [ 2 , 1 , -1 ]
[0.0, [1.0, 0.0]]
Matrice [ 1 , 1 , -1 ]
[-1.0, [1.0]]
Matrice [ 5 , 2 , -2 ]
[1.0, [0.0, 1.0, 0.0, 0.0, 0.0]]
Matrice [ 5 , 1 , -2 ]
[1.0, [1.0, 0.0, 0.0, 0.0, 0.0]]
Matrice [ 2 , 2 , 1 ]
[1.0, [1.0, 0.0]]
Matrice [ 1 , 2 , 1 ]
[0.0, [1.0]]
Matrice [ 4 , 3 , 0 ]
[1.0, [1.0, 0.0, 0.0, 0.0]]
Matrice [ 2 , 3 , 0 ]
[-1.0, [0.0, 1.0]]
Matrice [ 1 , 3 , 0 ]
[-1.0, [1.0]]
Matrice [ 6 , 4 , -1 ]
[1.0, [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
Matrice [ 3 , 2 , -1 ]
[0.0, [1.0, 0.0, 0.0]]
Matrice [ 2 , 2 , -1 ]
[-1.0, [1.0, 0.0]]
Matrice [ 1 , 2 , -1 ]
[-1.0, [1.0]]
Matrice [ 6 , 3 , -2 ]
[0.33333333, [0.33333333, 0.0, 0.666666

In [112]:
for i in range(NB_PIERRES_J2):
  
  v = getVal(23, i, -1)
  if v<0:
    print("Boucle for")
    print(v)

Matrice [ 23 , 1 , -1 ]
[1.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]]
Matrice [ 23 , 2 , -1 ]
[1.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]]
Matrice [ 23 , 3 , -1 ]
[1.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]]
Matrice [ 23 , 4 , -1 ]
[1.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
Matrice [ 23 , 5 , -1 ]
[1.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
Matrice [ 23 , 6 , -1 ]
[1.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
Matrice [ 23 , 7 , -1 ]
[1.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.



---

